# 3개의 노드

3개의 노드로 구성되어 있는 파이프라인을 작성합니다.<br>
예제는 세가지 케이스를 작성하여 확인하며 아래 그림과 같이 구성되어 있습니다.

<img src='node_example_image/node3.png'></img>

In [2]:
import kfp
from kfp import components, dsl
from kfp.components import func_to_container_op

## A -> B -> C

먼저 3개의 노드(컴포넌트)를 연결하기 위한 입력과 출력을 확인합니다.<br>
> nodeA : 입력은 선택적이지만 B의 입력을 받기위한 출력은 필수적입니다.<br>
> nodeB : A의 출력을 입력으로 받기 위해 A의 출력 형태를 정확히 알아야 하며 C의 입력을 받기위한 출력 또한 필수적입니다.<br>
> nodeC : B의 출력을 입력으로 받기 위해 B의 출력 형태를 정확히 알아야 합니다.

In [3]:
@func_to_container_op
def node_A() -> str:
    task_A = 'A'
    print(task_A)
    return task_A

@func_to_container_op
def node_B(B: str) -> str:
    task_B = f'{B} -> B'
    print(task_B)
    return task_B

@func_to_container_op
def node_C(C: str) -> str:
    task_C = f'{C} -> C'
    print(task_C)
    return task_C

컴포넌트를 모두 정의하였으면 입력과 출력을 연결하여 파이프라인을 작성합니다.

In [4]:
@dsl.pipeline(name='node3 example')
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.output)
    node_C_task = node_C(node_B_task.output)

### kubeflow pipeline 결과 화면

<img src='node_example_image/node3_1.png' width='850px'></img>

## (A , B) -> C

두 개의 노드 출력을 모두 입력으로 받는 노드를 작성합니다. <br>
> nodeA, nodeB : 입력은 선택적이지만 C의 입력을 받기위한 출력은 필수적입니다.<br>
> nodeC : 두 노드의 출력 형태를 정확히 알아야 하며 출력 값을 동시에 받기위한 형태를 구성해야 합니다.

In [5]:
@func_to_container_op
def node_A() -> str:
    task_A = 'A'
    print(task_A)
    return task_A

@func_to_container_op
def node_B() -> str:
    task_B = 'B'
    print(task_B)
    return task_B

노드A,B는 간단하게 구성되지만 노드C는 두 개의 인수를 입력받아야 합니다.

In [6]:
@func_to_container_op
def node_C(C1: str, C2: str) -> str:
    task_C = f'{(C1, C2)} -> C'
    print(task_C)
    return task_C

노드A,B의 출력을 입력으로 연결하여 파이프라인을 구성합니다.

In [7]:
@dsl.pipeline(name='node3 example')
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B()
    node_C_task = node_C(node_A_task.output, node_B_task.output)

### kubeflow pipeline 결과 화면

<img src='node_example_image/node3_2.png' width='850px'></img>

## A -> B, A -> C

두 개의 노드에 각각 입력 받도록 하는 노드A의 출력을 확인하여 작성합니다. <br>
> nodeA : 두 노드 각각의 입력을 받기 위한 출력을 구성해야 합니다.<br>
> nodeB, nodeC : A의 출력을 입력으로 받기위해 A의 출력 형태를 정확히 알아야 합니다.

In [8]:
from typing import NamedTuple
@func_to_container_op
def node_A() -> NamedTuple('Outputs', [('first', str), ('second', str)]):
    task_A = 'A'
    print(task_A)
    return (task_A, task_A)

두 개의 출력을 갖기 위한 NamedTuple 패키지를 활용합니다. 파이프라인을 구성하는 컴포넌트의 입력과 출력을 명확히 정의하기 위해 사용합니다.

In [9]:
@func_to_container_op
def node_B(B: str) -> str:
    task_B = f'{B} -> B'
    print(task_B)
    return task_B

@func_to_container_op
def node_C(C: str):
    task_C = f'{C} -> C'
    print(task_C)
    return task_C

노드A의 출력을 각각 입력받을 수 있는 노드 B,C를 정의한 후 파이프라인을 구성합니다. 파이프라인 구성 시 노드A의 출력은 2개 이상으로 구성되어 있기 때문에 key, value 형태로 다른 노드 입력에 작성하여 연결합니다.

In [10]:
@dsl.pipeline(name='node3 example')
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.outputs['first'])
    node_C_task = node_C(node_A_task.outputs['second'])

### kubeflow pipeline 결과 화면

<img src='node_example_image/node3_3.png' width='850px'></img>